In [2]:
#Ensemble Learning

In [3]:
#Initial Imports
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter  

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [5]:
# Load the data
file_path = Path('../Classification/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,RENT,26000.0,Source Verified,Jan-2019,low_risk,n,9.60,...,80.0,0.0,0.0,0.0,20625.0,6798.0,11300.0,5425.0,N,N
68813,12000.0,0.2727,368.37,RENT,63000.0,Not Verified,Jan-2019,low_risk,n,29.07,...,96.2,0.0,0.0,0.0,87939.0,60350.0,13500.0,62939.0,N,N
68814,5000.0,0.1992,185.62,MORTGAGE,52000.0,Source Verified,Jan-2019,low_risk,n,14.86,...,100.0,0.0,1.0,0.0,30592.0,18611.0,3600.0,18492.0,N,N
68815,40000.0,0.0646,1225.24,MORTGAGE,520000.0,Verified,Jan-2019,low_risk,n,9.96,...,98.2,12.5,0.0,0.0,1033574.0,95958.0,100800.0,78634.0,N,N


In [6]:
# Create our features
#X = df.drop(columns = 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', ''
X = df.drop(columns = 'loan_status')

# Create our target
y = df['loan_status']

In [7]:
X.describe() 

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [8]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [10]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [12]:
#Data Pre-Processing
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [31]:
X_train= X_train.select_dtypes(np.number)
X_test= X_test.select_dtypes(np.number)

In [32]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [33]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [34]:
#Ensemble Learners
#Balanced Random Forest Classifier

In [37]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=78)
rf_model = rf_model.fit(X_train_scaled, y_train)

In [39]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

In [41]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)


In [43]:
# Print the accuracy and imbalanced classification report
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,28,64
Actual 1,23,17090


Accuracy Score : 0.9949433304272014
Classification Report
              precision    recall  f1-score   support

   high_risk       0.55      0.30      0.39        92
    low_risk       1.00      1.00      1.00     17113

    accuracy                           0.99     17205
   macro avg       0.77      0.65      0.69     17205
weighted avg       0.99      0.99      0.99     17205



In [45]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.08338039743626877, 'total_rec_prncp'),
 (0.08076228335349345, 'revol_bal'),
 (0.07036001028203433, 'initial_list_status'),
 (0.06595544522094483, 'total_acc'),
 (0.053885410419416, 'out_prncp'),
 (0.021679069499172567, 'acc_open_past_24mths'),
 (0.020042330479286434, 'annual_inc'),
 (0.018732445047744538, 'all_util'),
 (0.016805037332721384, 'out_prncp_inv'),
 (0.016548867515985905, 'inq_last_12m'),
 (0.016205681977754287, 'pub_rec'),
 (0.01600808048206086, 'open_acc'),
 (0.015739052260633227, 'open_il_12m'),
 (0.015441207772164489, 'num_tl_30dpd'),
 (0.015362468872584554, 'delinq_2yrs'),
 (0.014779772458596019, 'num_tl_90g_dpd_24m'),
 (0.01472896406143695, 'total_rev_hi_lim'),
 (0.01471981833617401, 'last_pymnt_amnt'),
 (0.014414958318754096, 'num_tl_op_past_12m'),
 (0.014365360667346571, 'max_bal_bc'),
 (0.014247804084263599, 'installment'),
 (0.01423608666216288, 'tot_coll_amt'),
 (0.014097699579399962, 'mths_since_rcnt_il'),
 (0.014022815055153834, 'pct_tl_nvr_dlq'),
 (0.014008

In [49]:
#Easy Ensemble Classifier
from imblearn.ensemble import EasyEnsembleClassifier 

In [50]:
# Train the Classifier
rf_model2 = EasyEnsembleClassifier (n_estimators=100, random_state=78)
rf_model2 = rf_model2.fit(X_train_scaled, y_train)

In [51]:
# Making predictions using the testing data
predictions2 = rf_model2.predict(X_test_scaled)

In [52]:
cm2 = confusion_matrix(y_test, predictions2)
cm_df2 = pd.DataFrame(
    cm2, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score2 = accuracy_score(y_test, predictions2)

In [53]:
# Print the accuracy and imbalanced classification report
print("Confusion Matrix")
display(cm_df2)
print(f"Accuracy Score : {acc_score2}")
print("Classification Report")
print(classification_report(y_test, predictions2))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,54,38
Actual 1,3581,13532


Accuracy Score : 0.7896541702993316
Classification Report
              precision    recall  f1-score   support

   high_risk       0.01      0.59      0.03        92
    low_risk       1.00      0.79      0.88     17113

    accuracy                           0.79     17205
   macro avg       0.51      0.69      0.46     17205
weighted avg       0.99      0.79      0.88     17205



In [55]:
#Final Questions

#1. Which model had the best balanced accuracy score?
print('The Balanced Random Forest Classifier had the best balanced accuracy score.')

#2. Which model had the best recall score?
print('The Easy Ensemble Classifier had the better recall score.')

#3. Which model had the best geometric mean score?
print('The Balanced Random Forest Classifier had the best geometric score.')

#4. What are the top three features?
print('The top 3 features were: total_rec_prncp, revol_bal, and initial_list_status.')


The Balanced Random Forest Classifier had the best balanced accuracy score.
The Easy Ensemble Classifier had the better recall score.
The Balanced Random Forest Classifier had the best geometric score.
The top 3 features were: total_rec_prncp, revol_bal, and initial_list_status.
